# Post #2: First Contact - Making Your First Claude API Call

**Building a Credit Intelligence Platform - Part 2 of 10**

---

## What We're Building

In this notebook, we'll:
- Make our first Claude API call
- Build a simple credit risk analyzer
- Get structured JSON output (instead of plain text)
- Add error handling and cost tracking
- Create a reusable credit analysis function

**By the end:** You'll have a working credit risk analyzer that can evaluate borrowers and return structured recommendations.

## Prerequisites

- Completed Post #1 (API key configured)
- Anthropic API key set in environment (Critical!!!)

## Our Goals in this Workbook!

1. Basic API call structure
2. Understanding response objects
3. Getting structured output
4. Error handling patterns
5. Cost tracking
6. Production-ready code

---

## 1) Setup

First, let's install the required packages and set up our environment.

In [ ]:
# Test if you have the required packages installed, and install them if not.
try:    
    import anthropic 
    from dotenv import load_dotenv
    print("🌯 All packages already installed - you're good to go!")  # Add this
except ImportError:    
    print("🥙 Please install the required packages: anthropic, python-dotenv")
    print("🥙🥙   Run: pip install anthropic python-dotenv")  # Make it actionable
# NOTE: if you have the packages installed, you will see no output from the above code. 
# If you see an error message, it means you need to install the packages.


🌯 All packages already installed - you're good to go!


In [37]:
# Import libraries
import os
import json
from anthropic import Anthropic
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Initialize the Anthropic client
# This uses the ANTHROPIC_API_KEY from your environment (set in Post #1)
client = Anthropic(api_key=os.environ.get("ANTHROPIC_API_KEY"))

print("❤️ Client initialized successfully!")
print(f"❤️ API key found: {os.environ.get('ANTHROPIC_API_KEY')[:6]}...") # Print the first 6 characters of the API key to confirm it's loaded (don't print the whole key! We went through this in Post#1!☠️)

❤️ Client initialized successfully!
❤️ API key found: sk-ant...


---

## 2) Hello World: The Simplest API Call

Let's start with the absolute simplest API call to make sure everything works.

In [38]:
# Make the simplest possible API call
response = client.messages.create(
    model="claude-sonnet-4-20250514",  # Claude Sonnet 4.5
    #model = "claude-haiku-3-20240307"  # Cheapest model, good for testing
    #model = "claude-opus-4-20251101"  # Premium model, best for complex tasks

    max_tokens=1000,                     # Limit response length to testing
    messages=[
        {"role": "user", "content": "Hello! Can you help me analyze credit risk?"}
    ]
)

# Print the response
print(response.content[0].text)

I'd be happy to help you analyze credit risk! This is a broad area, so I can assist with various aspects depending on your specific needs.

Here are some key areas I can help with:

## Credit Risk Analysis Components

**1. Financial Analysis**
- Reviewing financial statements and ratios
- Cash flow analysis
- Debt service coverage ratios
- Liquidity and solvency metrics

**2. Risk Assessment Models**
- Probability of Default (PD) calculations
- Loss Given Default (LGD) estimation
- Exposure at Default (EAD) analysis
- Expected Loss calculations

**3. Credit Scoring & Rating**
- Developing scoring models
- Interpreting credit ratings
- Risk categorization methods

**4. Portfolio Analysis**
- Concentration risk assessment
- Diversification analysis
- Stress testing scenarios

**5. Industry/Sector Considerations**
- Industry-specific risk factors
- Economic sensitivity analysis
- Regulatory requirements

What specific aspect of credit risk analysis are you working on? Are you:
- Evaluatin

**What just happened?**

- We sent a message to Claude using `client.messages.create()`
- Claude responded with a helpful message
- We extracted the text using `response.content[0].text`

**Key parameters:**
- `model`: Which Claude model to use (Sonnet 4.5 is great for most tasks)
- `max_tokens`: Maximum length of response (~4 characters per token)
- `messages`: List of conversation turns (more on this later)

---

## 3) Understanding the Response Object

Let's examine what Claude actually returns. The response object contains more than just text - it has metadata we can use for debugging and cost tracking.

In [54]:
# Make a call and examine the full response
response = client.messages.create(
    model="claude-sonnet-4-20250514",
    max_tokens=100,
    messages=[{"role": "user", "content": "Hello!"}]
)

# Print all the fields
print("=" * 60)
print("RESPONSE OBJECT STRUCTURE")
print("=" * 60)
print(f"ID:           {response.id}")
print(f"Type:         {response.type}")
print(f"Role:         {response.role}")
print(f"Model:        {response.model}")
print(f"Stop Reason:  {response.stop_reason}")
print(f"\nContent:      {response.content}")
print(f"\nUsage:")
print(f"  Input tokens:  {response.usage.input_tokens}")
print(f"  Output tokens: {response.usage.output_tokens}")
print("=" * 60)

RESPONSE OBJECT STRUCTURE
ID:           msg_01F66XRgbnpxrW2QS42CMreh
Type:         message
Role:         assistant
Model:        claude-sonnet-4-20250514
Stop Reason:  end_turn

Content:      [TextBlock(citations=None, text="Hello! It's nice to meet you. How are you doing today? Is there anything I can help you with?", type='text')]

Usage:
  Input tokens:  9
  Output tokens: 27


**Understanding these fields:**

- **id**: Unique identifier for this message (useful for logging/debugging)
- **stop_reason**: Why Claude stopped generating
  - `end_turn`: Finished naturally
  - `max_tokens`: Hit the token limit
  - `stop_sequence`: Hit a custom stop sequence
- **content**: List of content blocks (text, but could include other types)
- **usage**: Token counts - **THIS IS IMPORTANT FOR COSTS!**

**Token Costs (as of January 2025):**
- Claude Sonnet 4.5: $3 per million input tokens, $15 per million output tokens
- This "Hello" exchange: ~10 input + 25 output tokens ≈ $0.00021

**💡 Pro tip:** Always track `usage` - it's cheap for testing but adds up at scale!

---

## 4) Building a Credit Risk Prompt (First Attempt)

Now let's make this actually useful. We'll create a function that analyzes credit risk for a borrower.

In [40]:
def analyze_credit_risk_v1(borrower_info: dict) -> str:
    """
    This is Version 1: Basic credit risk analysis. Returns unstructured text response. 
    NOTE: This is just playtime. In a real application, 
    you would want to structure the response more clearly, 
    and do more prompt engineering to get better results. 
    We will get to that in later posts. For now, let's just 
    see if we can get a response from the model based on 
    borrower information. ⁉️ -> 🫣
    """
    
    # Construct the prompt with borrower information
    prompt = f"""
    Analyze this borrower for credit risk:

    Name: {borrower_info['name']}
    Annual Income: ${borrower_info['income']:,}
    Credit Score: {borrower_info['credit_score']}
    Debt-to-Income: {borrower_info['dti']}%
    Employment: {borrower_info['employment']}
    Loan Type: {borrower_info['loan_type']}
    Loan Purpose: {borrower_info['purpose']}
    Loan Amount: ${borrower_info['loan_amount']:,}
    Collateral Value: ${borrower_info.get('collateral_value', 'N/A'):,} if borrower_info.get('collateral_value') else 'N/A (Unsecured)'
    LTV Ratio: {borrower_info.get('ltv', 'N/A')}% if borrower_info.get('ltv') else 'N/A'

    Should we approve this loan? Provide your analysis. Justify your decision based 
    on the borrower's financial profile and the loan details provided.
    """
    
    response = client.messages.create(
        model="claude-sonnet-4-20250514",
        max_tokens=500,
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.content[0].text

# Example 1: Mortgage with good LTV
borrower = {
    "name": "John Smith",
    "income": 85000,
    "credit_score": 720,
    "dti": 35,
    "employment": "Software Engineer, 5 years",
    "loan_type": "Mortgage",
    "purpose": "Home purchase",
    "loan_amount": 350000,
    "collateral_value": 400000,
    "ltv": 87.5
}

result = analyze_credit_risk_v1(borrower)
print(result)

Based on my analysis of John Smith's financial profile, I recommend **APPROVING** this mortgage loan with standard terms.

## Credit Risk Analysis

### **Positive Factors:**

**Strong Creditworthiness**
- Credit score of 720 is well above the conventional mortgage threshold (typically 620+)
- Indicates responsible payment history and credit management

**Stable Income Profile**
- Annual income of $85,000 is solid for mortgage qualification
- 5 years of employment as a Software Engineer demonstrates job stability
- Technology sector typically offers good earning potential and job security

**Reasonable Debt Load**
- Debt-to-income ratio of 35% is within acceptable limits (most lenders prefer <43%)
- Shows borrower can manage existing obligations while taking on mortgage payments

**Adequate Collateral Protection**
- Property value of $400,000 provides reasonable security for the $350,000 loan
- LTV ratio of 87.5% is manageable, though on the higher side

### **Areas of Consideration:**


**The Problem with V1:**

This gives us a good analysis, but:
- 😩 Format varies between calls
- 😩 Hard to parse programmatically
- 😩 Can't easily extract the recommendation
- 😩 Not consistent across different borrowers

**What we need:** Structured output that we can use in code, store in dbs, and build UIs around.

Let's fix this! 👇

---

## 5) Getting Structured Output (Version 2)

Instead of free-form text, let's get Claude to return a JSON object with a consistent structure.

In [41]:
def analyze_credit_risk_v2(borrower_info: dict) -> dict:
    """
    Version 2: Returns structured JSON output. W00t!
    NOTE: Okay, now let's get a bit more structured. Instead of just asking for an analysis,
    let's ask the model to return a JSON object with specific fields that we can easily parse and
    analyze programmatically. This is more useful for building an application where we want to
    automate decision-making or integrate with other systems. We will also ask for specific recommendations,
    identified risks, and suggested mitigations. This is a more realistic output format for a credit risk analysis tool.
    Now we go from 🫣 -> 🤔
    """
    
    prompt = f"""
    You are a credit risk analyst. Analyze this borrower and return your assessment 
    as a JSON object with the following structure:
    
    {{
        "recommendation": "APPROVE" | "APPROVE_WITH_CONDITIONS" | "DECLINE" | "REVIEW",
        "risk_level": "LOW" | "MEDIUM" | "HIGH",
        "key_risks": [list of identified risk factors],
        "mitigants": [list of suggested mitigations],
        "suggested_terms": {{
            "rate_adjustment": "adjustment in basis points or percentage",
            "required_down_payment": "minimum down payment percentage",
            "reserves_months": "months of reserves required"
        }},
        "confidence": numeric value between 0 and 1,
        "reasoning": "brief explanation of recommendation"
    }}
    
    Borrower Information:
    - Name: {borrower_info['name']}
    - Annual Income: ${borrower_info['income']:,}
    - Credit Score: {borrower_info['credit_score']}
    - Debt-to-Income: {borrower_info['dti']}%
    - Employment: {borrower_info['employment']}
    - Loan Purpose: {borrower_info['purpose']}
    - Loan Amount: ${borrower_info['loan_amount']:,}
    
    Return ONLY the JSON object, no additional text or markdown formatting.
    """
    
    response = client.messages.create(
        model="claude-sonnet-4-20250514",
        max_tokens=1000,
        messages=[{"role": "user", "content": prompt}]
    )
    
    # Extract the response text
    result_text = response.content[0].text
    
    # Claude sometimes wraps JSON in markdown code blocks - handle that
    if result_text.startswith("```json"):
        result_text = result_text.strip("```json").strip("```").strip()
    elif result_text.startswith("```"):
        result_text = result_text.strip("```").strip()
    
    # Parse the JSON
    return json.loads(result_text)

# Example 1: Mortgage with good LTV
borrower = {
    "name": "John Smith",
    "income": 85000,
    "credit_score": 720,
    "dti": 35,
    "employment": "Software Engineer, 5 years",
    "loan_type": "Mortgage",
    "purpose": "Home purchase",
    "loan_amount": 350000,
    "collateral_value": 400000,
    "ltv": 87.5
}

# Test it
result = analyze_credit_risk_v2(borrower)

# Pretty print the JSON
print(json.dumps(result, indent=2))


{
  "recommendation": "APPROVE_WITH_CONDITIONS",
  "risk_level": "MEDIUM",
  "key_risks": [
    "High loan-to-income ratio (4.1x annual income)",
    "Debt-to-income at upper acceptable range",
    "Large loan amount relative to income may strain cash flow"
  ],
  "mitigants": [
    "Strong employment history in stable field",
    "Good credit score indicates payment reliability",
    "Software engineering provides growth potential",
    "Require adequate reserves for payment security"
  ],
  "suggested_terms": {
    "rate_adjustment": "+25 basis points",
    "required_down_payment": "20%",
    "reserves_months": "4"
  },
  "confidence": 0.78,
  "reasoning": "Borrower has strong credit profile and stable employment, but loan amount is high relative to income. Conditions help mitigate cash flow risks while supporting homeownership goals."
}


**Why This Is Better:**

- 🌮 Consistent structure every time
- 🌮 Easy to parse and use in code
- 🌮 Can extract specific fields (e.g., just the recommendation)
- 🌮 Can store in databases with defined schema
- 🌮 Can build dashboards/UIs on top of this

NOW, let's try two more loan examples and see the output:

In [42]:
# Example 2: Auto loan
borrower= {
    "name": "Jane Doe",
    "income": 120000,
    "credit_score": 780,
    "dti": 28,
    "employment": "Doctor, 8 years",
    "loan_type": "Auto",
    "purpose": "Vehicle purchase",
    "loan_amount": 45000,
    "collateral_value": 50000,
    "ltv": 90.0
}

# Test it
result = analyze_credit_risk_v2(borrower)

# Pretty print the JSON
print(json.dumps(result, indent=2))


{
  "recommendation": "APPROVE",
  "risk_level": "LOW",
  "key_risks": [
    "Vehicle depreciation risk",
    "Single income source dependency"
  ],
  "mitigants": [
    "Stable employment in essential profession",
    "Excellent credit history demonstrates payment reliability",
    "Conservative debt-to-income ratio provides payment cushion",
    "Strong income level relative to loan amount"
  ],
  "suggested_terms": {
    "rate_adjustment": "0 basis points",
    "required_down_payment": "10%",
    "reserves_months": "2"
  },
  "confidence": 0.92,
  "reasoning": "Excellent credit profile with strong income, low DTI, and stable employment in essential profession. Vehicle loan amount is reasonable relative to income. Minimal risk factors present."
}


In [43]:
# Example 3: Unsecured personal loan
borrower= {
    "name": "Bob Johnson",
    "income": 55000,
    "credit_score": 650,
    "dti": 42,
    "employment": "Retail Manager, 2 years",
    "loan_type": "Personal",
    "purpose": "Debt consolidation",
    "loan_amount": 35000,
    "collateral_value": None,
    "ltv": None
}

# Test it
result = analyze_credit_risk_v2(borrower)

# Pretty print the JSON
print(json.dumps(result, indent=2))

{
  "recommendation": "APPROVE_WITH_CONDITIONS",
  "risk_level": "MEDIUM",
  "key_risks": [
    "High debt-to-income ratio at 42%",
    "Fair credit score of 650 indicates past credit management issues",
    "Retail industry employment may have income volatility",
    "Relatively short employment tenure of 2 years",
    "Large loan amount relative to annual income (64% of gross income)"
  ],
  "mitigants": [
    "Debt consolidation purpose should improve overall debt management",
    "Stable employment as manager indicates responsibility",
    "Income level supports loan payments if DTI improves post-consolidation",
    "Require proof of debt payoff to ensure DTI improvement"
  ],
  "suggested_terms": {
    "rate_adjustment": "+150 basis points above prime rate",
    "required_down_payment": "Not applicable for debt consolidation",
    "reserves_months": "3 months of loan payments in reserves"
  },
  "confidence": 0.72,
  "reasoning": "Borrower presents moderate risk due to high DTI an

**Now we can do things like:**

In [44]:
# Extract just the recommendation
print(f"Recommendation: {result['recommendation']}")
print(f"Risk Level: {result['risk_level']}")
print(f"Confidence: {result['confidence']}")

# Check if approval is needed
if result['recommendation'] in ['APPROVE', 'APPROVE_WITH_CONDITIONS']:
    print("\n🤑 This loan can proceed to closing!")
    if result['suggested_terms']['required_down_payment']:
        print(f"Required down payment: {result['suggested_terms']['required_down_payment']}")
else:
    print("\n⛔ This loan requires review or decline")

Recommendation: APPROVE_WITH_CONDITIONS
Risk Level: MEDIUM
Confidence: 0.72

🤑 This loan can proceed to closing!
Required down payment: Not applicable for debt consolidation


---

## 6) Error Handling (The Unglamorous Necessity)

APIs fail. Networks timeout. JSON parsing breaks. Let's add proper error handling.

In [45]:
def analyze_credit_risk_v3(borrower_info: dict) -> dict:
    """
    Version 3: Adds comprehensive error handling.
    NOTE: In a real application, you will want to add robust error handling to manage 
    cases where the model's response is not in the expected format, or when there are 
    issues with the API call. This version includes error handling for JSON parsing and 
    checks for required fields in the response. It also logs token usage for cost tracking. 
    This is important for building a production-ready application that can handle edge 
    cases gracefully(?). Now we go from 🤔 -> 😎 or 🤓 (Both are c00l)
    """
    
    try:
        # Build the prompt
        prompt = f"""
        You are a credit risk analyst. Analyze this borrower and return your assessment 
        as a JSON object with the following structure:
        
        {{
            "recommendation": "APPROVE" | "APPROVE_WITH_CONDITIONS" | "DECLINE" | "REVIEW",
            "risk_level": "LOW" | "MEDIUM" | "HIGH",
            "key_risks": [list of identified risk factors],
            "mitigants": [list of suggested mitigations],
            "suggested_terms": {{
                "rate_adjustment": "adjustment",
                "required_down_payment": "percentage",
                "reserves_months": number
            }},
            "confidence": number between 0 and 1,
            "reasoning": "brief explanation"
        }}
        
        Borrower Information:
        - Name: {borrower_info['name']}
        - Annual Income: ${borrower_info['income']:,}
        - Credit Score: {borrower_info['credit_score']}
        - Debt-to-Income: {borrower_info['dti']}%
        - Employment: {borrower_info['employment']}
        - Loan Purpose: {borrower_info['purpose']}
        - Loan Amount: ${borrower_info['loan_amount']:,}
        
        Return ONLY valid JSON, no markdown formatting.
        """
        
        # Make API call
        response = client.messages.create(
            model="claude-sonnet-4-20250514",
            max_tokens=1000,
            messages=[{"role": "user", "content": prompt}]
        )
        
        # Log token usage for cost tracking
        print(f"Tokens used - Input: {response.usage.input_tokens}, "
              f"Output: {response.usage.output_tokens}")
        
        # Extract response
        result_text = response.content[0].text
        
        # Handle markdown wrapper
        if result_text.startswith("```"):
            result_text = result_text.strip("```json").strip("```").strip()
        
        # Parse JSON
        result = json.loads(result_text)
        
        # Validate required fields
        required_fields = ["recommendation", "risk_level", "confidence"]
        missing = [f for f in required_fields if f not in result]
        if missing:
            raise ValueError(f"Missing required fields: {missing}")
        
        return result
        
    except json.JSONDecodeError as e:
        print(f"😞 Failed to parse JSON response: {e}")
        print(f"Raw response: {result_text[:200]}...")  # Show first 200 chars
        raise
        
    except Exception as e:
        print(f"😩 Analysis failed: {e}")
        raise



In [46]:

# Test it with one of the borrowers (you can test with all of them to see how it handles different profiles)
try:
    result = analyze_credit_risk_v3(borrower)
    print("\n🎉 Analysis successful!")
    print(json.dumps(result, indent=2))
except Exception as e:
    print(f"\n😩 Analysis failed: {e}")


Tokens used - Input: 278, Output: 275

🎉 Analysis successful!
{
  "recommendation": "APPROVE_WITH_CONDITIONS",
  "risk_level": "MEDIUM",
  "key_risks": [
    "High debt-to-income ratio at 42%",
    "Fair credit score of 650 indicates past credit issues",
    "Relatively short employment history of 2 years",
    "High loan amount relative to income (64% of annual income)",
    "Retail sector employment may have income volatility"
  ],
  "mitigants": [
    "Debt consolidation purpose should improve overall financial position",
    "Stable employment as manager indicates responsibility",
    "Income level supports loan payments if DTI improves post-consolidation"
  ],
  "suggested_terms": {
    "rate_adjustment": "+2.5% above prime rate",
    "required_down_payment": "15%",
    "reserves_months": 3
  },
  "confidence": 0.72,
  "reasoning": "Borrower shows mixed risk profile with elevated DTI but stable employment. Debt consolidation purpose is positive, but credit score and DTI require ca

**What we added:**

- 🌮 Try-catch blocks for all error types
- 🌮 Token usage logging (for cost tracking)
- 🌮 JSON validation (check for required fields)
- 🌮 Helpful error messages with context
- 🌮 Graceful handling of markdown wrappers

**Common errors you'll encounter:**
1. **JSONDecodeError**: Claude returns invalid JSON → Better prompt instructions
2. **APIConnectionError**: Network issues → Add retry logic
3. **RateLimitError**: Too many requests → Implement rate limiting
4. **InvalidRequestError**: Malformed parameters → Validate inputs

---

## 7) Cost Tracking

Let's build a simple cost tracker so we know exactly how much we're spending.

In [47]:
class CostTracker:
    """Track API costs across multiple calls.
    This class keeps a running total of input and 
    output tokens, and calculates costs based on the 
    pricing for the model being used.
    Realistically, you would want to persist this data 
    to a database or file. This is just a simple in-memory 
    tracker for demonstration purposes.
    We already hit the c00l mark (😎 // 🤓), so let's keep going! """
    
    def __init__(self):
        self.total_input_tokens = 0
        self.total_output_tokens = 0
        
        # Sonnet 4.5 pricing (as of January 2026)
        self.input_cost_per_million = 3.00
        self.output_cost_per_million = 15.00
    
    def add_usage(self, usage):
        #Add token usage from a response.
        self.total_input_tokens += usage.input_tokens
        self.total_output_tokens += usage.output_tokens
    
    def get_cost(self) -> dict:
        #Calculate total cost so far.
        input_cost = (self.total_input_tokens / 1_000_000) * self.input_cost_per_million
        output_cost = (self.total_output_tokens / 1_000_000) * self.output_cost_per_million
        
        return {
            "input_tokens": self.total_input_tokens,
            "output_tokens": self.total_output_tokens,
            "input_cost": input_cost,
            "output_cost": output_cost,
            "total_cost": input_cost + output_cost
        }
    
    def print_summary(self):
        #Print cost summary.
        costs = self.get_cost()
        print("\n" + "=" * 50)
        print("API COST SUMMARY")
        print("=" * 50)
        print(f"Input tokens:  {costs['input_tokens']:,}")
        print(f"Output tokens: {costs['output_tokens']:,}")
        print(f"Input cost:    ${costs['input_cost']:.4f}")
        print(f"Output cost:   ${costs['output_cost']:.4f}")
        print(f"Total cost:    ${costs['total_cost']:.4f}")
        print("=" * 50 + "\n")

# Test the cost tracker
tracker = CostTracker()

# Make a few API calls
response = client.messages.create(
    model="claude-sonnet-4-20250514",
    max_tokens=100,
    messages=[{"role": "user", "content": "Hello!"}]
)
tracker.add_usage(response.usage)

# Make another call
response = client.messages.create(
    model="claude-sonnet-4-20250514",
    max_tokens=100,
    messages=[{"role": "user", "content": "What's 2+2?"}]
)
tracker.add_usage(response.usage)

# Print summary
tracker.print_summary()


API COST SUMMARY
Input tokens:  23
Output tokens: 40
Input cost:    $0.0001
Output cost:   $0.0006
Total cost:    $0.0007



**Why cost tracking matters:**

- Testing feels free, but costs add up
- Production usage can surprise you
- Helps optimize prompts (shorter = cheaper)
- Informs business decisions (cost per analysis)

**Cost optimization tips:**
1. **Shorter prompts** - Every word costs money
2. **Lower max_tokens** - Don't request more than needed
3. **Batch processing** - Group related analyses
4. **Caching** - Don't re-analyze same borrower

---

## 8) Building a Production-Ready Analyzer

Let's combine everything into a reusable class that's ready for real use.

In [48]:
from typing import Dict, Optional

class CreditAnalyzer:
    """
    LETS GO: Production-ready credit risk analyzer using Claude API.
    Handles structured analysis with error handling, cost tracking and
    is certified c00l (😎 // 🤓). 
    """
    
    def __init__(self, track_costs: bool = True):
        """Initialize the ANALYZER."""
        self.client = Anthropic(api_key=os.environ.get("ANTHROPIC_API_KEY"))
        self.model = "claude-sonnet-4-20250514"
        self.cost_tracker = CostTracker() if track_costs else None
    
    def analyze(self, borrower_info: Dict) -> Dict:
        """
        This is what it's all about:
        -> FIRST: 
        Analyze a borrower's credit risk.
        -> SECOND:
        Args:
            borrower_info: Dictionary with borrower details
        -> THIRD:    
        Returns:
            Structured analysis with recommendation
        """
        try:
            prompt = self._build_prompt(borrower_info)
            
            response = self.client.messages.create(
                model=self.model,
                max_tokens=1000,
                messages=[{"role": "user", "content": prompt}]
            )
            
            # Track costs if enabled
            if self.cost_tracker:
                self.cost_tracker.add_usage(response.usage)
            
            result = self._parse_response(response.content[0].text)
            return result
            
        except Exception as e:
            print(f"😩 Analysis failed for {borrower_info.get('name', 'Unknown')}: {e}")
            raise
    
    def _build_prompt(self, borrower_info: Dict) -> str:
        #NOW...Construct the credit analysis prompt!
        return f"""
        You are a credit risk analyst. Analyze this borrower and return a JSON object:
        
        {{
            "recommendation": "APPROVE" | "APPROVE_WITH_CONDITIONS" | "DECLINE" | "REVIEW",
            "risk_level": "LOW" | "MEDIUM" | "HIGH",
            "key_risks": [list of risk factors],
            "mitigants": [list of mitigations],
            "suggested_terms": {{
                "rate_adjustment": "rate adjustment",
                "required_down_payment": "down payment %",
                "reserves_months": months
            }},
            "confidence": 0.0 to 1.0,
            "reasoning": "brief explanation"
        }}
        
        Borrower: {borrower_info['name']}
        Income: ${borrower_info['income']:,}
        Credit Score: {borrower_info['credit_score']}
        DTI: {borrower_info['dti']}%
        Employment: {borrower_info['employment']}
        Purpose: {borrower_info['purpose']}
        Loan Amount: ${borrower_info['loan_amount']:,}
        
        Return ONLY valid JSON, no markdown.
        """
    
    def _parse_response(self, text: str) -> Dict:
        #Parse and validate the JSON response.
        # Remove markdown if present
        if text.startswith("```"):
            text = text.strip("```json").strip("```").strip()
        
        result = json.loads(text)
        
        # Validate required fields
        required = ["recommendation", "risk_level", "confidence"]
        missing = [f for f in required if f not in result]
        if missing:
            raise ValueError(f"Missing required fields: {missing}")
        
        return result
    
    def get_costs(self) -> Optional[Dict]:
        #Get cost summary if tracking is enabled.
        if self.cost_tracker:
            return self.cost_tracker.get_cost()
        return None
    
    def print_cost_summary(self):
        #Print cost summary.
        if self.cost_tracker:
            self.cost_tracker.print_summary()
        else:
            print("Cost tracking is disabled")

print("🎉 CreditAnalyzer class defined successfully!")

🎉 CreditAnalyzer class defined successfully!


**Why this design is better:**

- 🌮 **Encapsulated**: All logic in one class
- 🌮 **Reusable**: Import and use anywhere
- 🌮 **Testable**: Can mock the API client
- 🌮 **Observable**: Built-in cost tracking
- 🌮 **Maintainable**: Clear separation of concerns

Now let's use it!

In [49]:
# Create the analyzer
analyzer = CreditAnalyzer(track_costs=True)

# Define some test borrowers
borrowers = [
    {
        "name": "John Smith",
        "income": 85000,
        "credit_score": 720,
        "dti": 35,
        "employment": "Software Engineer, 5 years",
        "purpose": "Home purchase",
        "loan_amount": 350000
    },
    {
        "name": "Jane Doe",
        "income": 120000,
        "credit_score": 780,
        "dti": 28,
        "employment": "Doctor, 8 years",
        "purpose": "Home purchase",
        "loan_amount": 500000
    },
    {
        "name": "Bob Johnson",
        "income": 55000,
        "credit_score": 650,
        "dti": 42,
        "employment": "Retail Manager, 2 years",
        "purpose": "Auto loan",
        "loan_amount": 35000
    }
]

# Analyze each borrower
print("\n" + "=" * 60)
print("CREDIT ANALYSIS RESULTS")
print("=" * 60)

for borrower in borrowers:
    result = analyzer.analyze(borrower)
    
    print(f"\n📋 {borrower['name']}")
    print(f"   Recommendation: {result['recommendation']}")
    print(f"   Risk Level: {result['risk_level']}")
    print(f"   Confidence: {result['confidence']}")
    
    if result.get('key_risks'):
        print(f"   Key Risks: {', '.join(result['key_risks'][:2])}...")  # Show first 2

# Print cost summary
analyzer.print_cost_summary()


CREDIT ANALYSIS RESULTS

📋 John Smith
   Recommendation: APPROVE_WITH_CONDITIONS
   Risk Level: MEDIUM
   Confidence: 0.75
   Key Risks: High loan-to-income ratio (4.1x annual income), DTI at upper acceptable range (35%)...

📋 Jane Doe
   Recommendation: APPROVE
   Risk Level: LOW
   Confidence: 0.88
   Key Risks: High loan amount relative to income (4.2x income ratio), Potential income volatility in medical profession...

📋 Bob Johnson
   Recommendation: APPROVE_WITH_CONDITIONS
   Risk Level: MEDIUM
   Confidence: 0.75
   Key Risks: High debt-to-income ratio at 42%, Fair credit score of 650 indicates past credit challenges...

API COST SUMMARY
Input tokens:  745
Output tokens: 751
Input cost:    $0.0022
Output cost:   $0.0113
Total cost:    $0.0135



**Perfect!** Now we have a production-ready credit analyzer that:

- Analyzes multiple borrowers consistently
- Tracks costs automatically
- Returns structured data we can use programmatically
- Handles errors gracefully
- Can be easily integrated into larger systems

---

## 9) Experiments & Challenges

Now it's your turn to experiment! Try these challenges:

### Challenge 1: Modify a Borrower

Change John Smith's credit score to 650 and DTI to 45%. How does Claude's recommendation change?

In [50]:
# Your code here
modified_borrower = {
    "name": "John Smith (Modified)",
    "income": 85000,
    "credit_score": 650,  # Changed from 720
    "dti": 45,            # Changed from 35
    "employment": "Software Engineer, 5 years",
    "purpose": "Home purchase",
    "loan_amount": 350000
}

# Analyze and compare
result = analyzer.analyze(modified_borrower)
print(json.dumps(result, indent=2))

{
  "recommendation": "APPROVE_WITH_CONDITIONS",
  "risk_level": "MEDIUM",
  "key_risks": [
    "Credit score below prime threshold (650)",
    "High debt-to-income ratio at 45%",
    "Large loan amount relative to income (4.1x multiplier)",
    "Limited financial cushion after debt obligations"
  ],
  "mitigants": [
    "Stable employment history (5 years)",
    "Software engineering profession with growth potential",
    "Strong income level at $85,000",
    "Home purchase provides collateral security"
  ],
  "suggested_terms": {
    "rate_adjustment": "+0.75% above standard rate",
    "required_down_payment": "15%",
    "reserves_months": 4
  },
  "confidence": 0.78,
  "reasoning": "Borrower shows employment stability and adequate income, but elevated DTI and fair credit score require protective terms. The profession and income level support repayment capacity with appropriate conditions."
}


### Challenge 2: Add Input Validation

Add a function that validates borrower data before analysis:
- Credit score must be 300-850
- DTI must be 0-100
- Income must be positive
- All required fields present

In [51]:
def validate_borrower(borrower: dict) -> bool:
    """
    Validate borrower data before analysis.
    Returns True if valid, raises ValueError if not.
    NOTE: In a real application, you would want to validate 
    the input data before sending it to the model. We'll tackle 
    this down  the line. This function checks for required 
    fields, validates ranges for credit score and DTI, and 
    ensures that income and loan amount are positive. You 
    can expand this with more complex validation rules as 
    needed. This is an important step to ensure that your 
    application can handle bad data gracefully and provide 
    useful feedback to users. 
    
    Now we go from 😎 // 🤓 -> 🦾 (Super c00l and strong!💪) CYB0RG!
    """
    # Required fields
    required_fields = ['name', 'income', 'credit_score', 'dti', 
                       'employment', 'loan_type', 'purpose', 'loan_amount']
    
    # Check all required fields present
    missing = [f for f in required_fields if f not in borrower]
    if missing:
        raise ValueError(f"Missing required fields: {missing}")
    
    # Validate credit score range (300-850)
    if not 300 <= borrower['credit_score'] <= 850:
        raise ValueError(f"Credit score {borrower['credit_score']} outside valid range (300-850)")
    
    # Validate DTI range (0-100)
    if not 0 <= borrower['dti'] <= 100:
        raise ValueError(f"DTI {borrower['dti']}% outside valid range (0-100)")
    
    # Validate income is positive
    if borrower['income'] <= 0:
        raise ValueError(f"Income must be positive, got {borrower['income']}")
    
    # Validate loan amount is positive
    if borrower['loan_amount'] <= 0:
        raise ValueError(f"Loan amount must be positive, got {borrower['loan_amount']}")
    
    # If LTV is provided, validate it
    if 'ltv' in borrower and borrower['ltv'] is not None:
        if not 0 < borrower['ltv'] <= 125:  # Some lenders go above 100%
            raise ValueError(f"LTV {borrower['ltv']}% outside reasonable range (0-125)")
    
    # If collateral_value provided, validate it's positive
    if 'collateral_value' in borrower and borrower['collateral_value'] is not None:
        if borrower['collateral_value'] <= 0:
            raise ValueError(f"Collateral value must be positive")
    
    return True

# Test it
try:
    validate_borrower(borrower)
    print("🎉 Validation passed!")
except ValueError as e:
    print(f"😩 Validation failed: {e}")

# Test with bad data
bad_borrower = {
    "name": "Invalid",
    "income": -50000,  # Negative!
    "credit_score": 900,  # Too high!
    "dti": 35,
    "employment": "Test",
    "loan_type": "Mortgage",
    "purpose": "Test",
    "loan_amount": 100000
}

try:
    validate_borrower(bad_borrower)
except ValueError as e:
    print(f"🎉 Correctly caught error: {e}")

😩 Validation failed: Missing required fields: ['loan_type']
🎉 Correctly caught error: Credit score 900 outside valid range (300-850)


### Challenge 3: Batch Processing

Modify the analyzer to process a list of borrowers and return a summary:
- How many approved vs declined?
- Average confidence score
- Most common risk factors

In [55]:
def batch_analyze(borrowers: list) -> dict:
    """
    Analyze multiple borrowers and return summary statistics.
    """
    from collections import Counter
    
    # Initialize tracking
    results = []
    all_risks = []
    
    # Analyze each borrower
    analyzer = CreditAnalyzer(track_costs=True)
    
    for borrower in borrowers:
        try:
            result = analyzer.analyze(borrower)
            result['borrower_name'] = borrower['name']  # Add name for reference
            results.append(result)
            
            # Collect risks for frequency analysis
            if 'key_risks' in result:
                all_risks.extend(result['key_risks'])
                
        except Exception as e:
            print(f"Failed to analyze {borrower.get('name', 'Unknown')}: {e}")
            continue
    
    # Calculate summary statistics
    recommendations = [r['recommendation'] for r in results]
    rec_counts = Counter(recommendations)
    
    risk_levels = [r['risk_level'] for r in results]
    risk_counts = Counter(risk_levels)
    
    confidences = [r['confidence'] for r in results]
    avg_confidence = sum(confidences) / len(confidences) if confidences else 0
    
    # Most common risks
    risk_frequency = Counter(all_risks)
    top_risks = risk_frequency.most_common(5)
    
    # Get costs
    costs = analyzer.get_costs()
    
    summary = {
        "total_analyzed": len(results),
        "recommendations": dict(rec_counts),
        "risk_levels": dict(risk_counts),
        "average_confidence": round(avg_confidence, 3),
        "top_risk_factors": [{"risk": risk, "frequency": count} for risk, count in top_risks],
        "total_cost": costs['total_cost'] if costs else 0,
        "cost_per_analysis": costs['total_cost'] / len(results) if costs and results else 0,
        "detailed_results": results  # Include all individual results
    }
    
    return summary

# Test it
borrowers = [
    {
        "name": "John Smith",
        "income": 85000,
        "credit_score": 720,
        "dti": 35,
        "employment": "Software Engineer, 5 years",
        "loan_type": "Mortgage",
        "purpose": "Home purchase",
        "loan_amount": 350000,
        "collateral_value": 400000,
        "ltv": 87.5
    },
    {
        "name": "Jane Doe",
        "income": 120000,
        "credit_score": 780,
        "dti": 28,
        "employment": "Doctor, 8 years",
        "loan_type": "Mortgage",
        "purpose": "Home purchase",
        "loan_amount": 500000,
        "collateral_value": 625000,
        "ltv": 80.0
    },
    {
        "name": "Bob Johnson",
        "income": 55000,
        "credit_score": 650,
        "dti": 42,
        "employment": "Retail Manager, 2 years",
        "loan_type": "Personal",
        "purpose": "Debt consolidation",
        "loan_amount": 35000,
        "collateral_value": None,
        "ltv": None
    },
    {
        "name": "Alice Williams",
        "income": 95000,
        "credit_score": 690,
        "dti": 38,
        "employment": "Teacher, 10 years",
        "loan_type": "Auto",
        "purpose": "Vehicle purchase",
        "loan_amount": 40000,
        "collateral_value": 45000,
        "ltv": 88.9
    }
]

# Run batch analysis
summary = batch_analyze(borrowers)

# Pretty print results
print("\n" + "="*60)
print("BATCH ANALYSIS SUMMARY")
print("="*60)
print(f"\nTotal Analyzed: {summary['total_analyzed']}")
print(f"\nRecommendations:")
for rec, count in summary['recommendations'].items():
    print(f"  {rec}: {count}")

print(f"\nRisk Levels:")
for risk, count in summary['risk_levels'].items():
    print(f"  {risk}: {count}")

print(f"\nAverage Confidence: {summary['average_confidence']:.1%}")

print(f"\nTop Risk Factors:")
for item in summary['top_risk_factors']:
    print(f"  {item['risk']}: {item['frequency']} occurrences")

print(f"\nCost Analysis:")
print(f"  Total Cost: ${summary['total_cost']:.4f}")
print(f"  Cost Per Analysis: ${summary['cost_per_analysis']:.4f}")
print("="*60)


BATCH ANALYSIS SUMMARY

Total Analyzed: 4

Recommendations:
  APPROVE_WITH_CONDITIONS: 3
  APPROVE: 1

Risk Levels:
  MEDIUM: 3
  LOW: 1

Average Confidence: 78.7%

Top Risk Factors:
  High loan-to-income ratio (4.1x annual income): 1 occurrences
  DTI at 35% approaches upper acceptable threshold: 1 occurrences
  Single income source dependency: 1 occurrences
  No information on liquid assets or down payment amount: 1 occurrences
  High loan amount requiring substantial income verification: 1 occurrences

Cost Analysis:
  Total Cost: $0.0185
  Cost Per Analysis: $0.0046


---

## 🎓 What You Learned

In this notebook, you:

1. 🌮 Made your first Claude API call
2. 🌮 Understood response object structure
3. 🌮 Built structured prompts for credit analysis
4. 🌮 Got JSON output instead of unstructured text
5. 🌮 Added error handling for production use
6. 🌮 Implemented cost tracking
7. 🌮 Created a reusable `CreditAnalyzer` class
8. 🌮 Analyzed real borrower scenarios

## 🚀 What's Next

**Post #3: Advanced Prompt Engineering**
- Making prompts bulletproof
- Few-shot learning with examples
- Intent detection and routing
- Cost optimization techniques
- Consistency and reliability patterns

**The Full Series:**
- Posts 1-3: API fundamentals 🌮
- Posts 4-5: RAG system (credit knowledge base)
- Posts 6-7: MCP integration (live data)
- Posts 8-9: Multi-agent orchestration
- Post 10: Production deployment

## 📚 Resources

- [Read the full blog post](link-to-medium)
- [Anthropic API Documentation](https://docs.anthropic.com)
- [GitHub Repository](your-repo-link)


GET PUMPED! 💪🏼